## 1. Libraries

In [1]:
import numpy as np
import pandas as pd

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State

import plotly.graph_objects as go
import plotly.offline as pyo

In [5]:
## Template
app = dash.Dash()

app.layout = html.Div([
    html.Div([]),
    html.Div([])
], style={})


#@app.callback(Output(),
#              [Input()],
#              [State()])
def func():
    pass
    return ""

if __name__ == "__main__":
    pass
    #app.run_server()

## 2. Stock Ticker App
Resources:
- [Guidebook](https://docs.google.com/document/d/1O76ws8azib9Nf6pYlqf_4w8jklBfdRKvh-WwVL1mcmE/edit)
- [HTML Components](https://dash.plot.ly/dash-html-components)
- [Core Components](https://dash.plot.ly/dash-core-components)
- [Markdown](https://dash.plot.ly/dash-core-components/markdown)
- [State](https://dash.plot.ly/state)
- [Interactive Graphs](https://dash.plot.ly/interactive-graphing)
- [Callbacks](https://dash.plot.ly/getting-started-part-2)

In [13]:
import pandas_datareader.data as web
from datetime import datetime

In [33]:
nsdq = pd.read_csv("../Data/NASDAQcompanylist.csv")
nsdq.set_index("Symbol", inplace=True)
nsdq.head(2)

,Name,LastSale,MarketCap,IPOyear,Sector,Industry
Symbol,,,,,,
ABMD,"ABIOMED, Inc.",286.22,1.267320e+10,NaN,Health Care,Medical/Dental Instruments
ATVI,"Activision Blizzard, Inc",64.56,4.897726e+10,NaN,Technology,Computer Software: Prepackaged Software


In [39]:
option = []
for tic in nsdq.index:
    my_dict = {}
    my_dict["label"] = str(nsdq.loc[tic,"Name"]) + " " + tic
    my_dict["value"] = tic
    option.append(my_dict)

In [43]:
app = dash.Dash()

app.layout = html.Div([
    html.H1("Stock Ticker Dashboard"),
    html.Div([html.H3("Enter a stock symbol", style={"paddingRight":"30px"}), 
              dcc.Dropdown(id="my-stock-picker", 
                           options=option,value=["TSLA"], 
                           multi=True)],
             style={"display":"inline-block","verticalAlign":"top", "width":"30%"}),
    html.Div([html.H3("Enter a start and end date", style={"paddingRight":"30px"}),
              dcc.DatePickerRange(id="my-date-picker", 
                                  min_date_allowed=datetime(2017,1,1), 
                                  max_date_allowed=datetime.today(), 
                                  start_date=datetime(2019,1,1), 
                                  end_date=datetime.today())], 
              style={"display":"inline-block"}),
    html.Div([html.Button(id="submit-button", n_clicks=0, 
                          children="Submit", 
                          style={"fontSize":24,"marginLeft":"30px"})],
             style={"display":"inline-block"}),
    
    html.Div([dcc.Graph(id="my-graph", 
                        figure={"data":[{"x":[1,2],
                                         "y":[3,1]}],
                                "layout":{"title":"Default Title"}})])
], style={})

@app.callback(Output("my-graph","figure"),
              [Input("submit-button","n_clicks")],
              [State("my-stock-picker","value"), 
               State("my-date-picker","start_date"), 
               State("my-date-picker","end_date")])

def update_graph(n_clicks,stock_picker, start_date, end_date):
    traces = []
    start = datetime.strptime(start_date[:10], "%Y-%m-%d")
    end = datetime.strptime(end_date[:10], "%Y-%m-%d")
    for tic in stock_picker:
        df = web.DataReader(tic, "iex", start, end)
        traces.append({"x":df.index,"y":df["close"]})
        
    fig = {"data":traces,
           "layout":{"title":stock_picker}}
    return fig

if __name__ == "__main__":
    pass
    #app.run_server()